# Test Notebook: Executing optimizer.py and capacity_optimizer.py (step-by-step)

This notebook runs the raw functions with detailed prints: recursive route generation, PuLP LP formulation output, and step-by-step data transformations.

In [1]:
import sys
import os
import pandas as pd
from datetime import datetime, timedelta
from pulp import *
import logging

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)
print('✓ Imports OK - Python', sys.version.split()[0])

✓ Imports OK - Python 3.10.0


In [2]:
VEHICLE_SPECS = {
    'truck': {'speed': 80, 'cost': 0.50},
    'van': {'speed': 90, 'cost': 0.40},
    'car': {'speed': 100, 'cost': 0.30},
    'auto': {'speed': 60, 'cost': 0.20},
    'bike': {'speed': 25, 'cost': 0.10},
    'plane': {'speed': 900, 'cost': 2.00},
}
print('✓ VEHICLE_SPECS loaded')

✓ VEHICLE_SPECS loaded


In [5]:
def enumerate_paths(nodes, edges, start, end, max_depth=10):
    """Recursively enumerate simple paths from start to end and print recursion steps.
    Returns list of paths (each path is list of nodes).
    """
    paths = []
    def dfs(current, path, depth):
        print(f"[RECURSION] depth={depth} current={current} path={path}")
        if depth > max_depth:
            print('  Max depth reached, backtracking')
            return
        if current == end:
            print(f'  Found path: {path}')
            paths.append(list(path))
            return
        for (u, v) in edges.keys():
            if u == current and v not in path:
                path.append(v)
                dfs(v, path, depth+1)
                path.pop()
    dfs(start, [start], 0)
    print(f"Total enumerated paths: {len(paths)}")
    return paths

print('✓ enumerate_paths defined')

✓ enumerate_paths defined


In [6]:
def optimize(nodes, edges, start, end, objective='cost', via=None, lp_filename='model.lp', solver_msg=True):
    """Build Lp problem, print full formulation, write LP file, enumerate paths, and solve with CBC.
    Also prints selection of edges and solver output.
    """
    print('\n[INPUT]')
    print('  start=', start, ' end=', end, ' objective=', objective, ' via=', via)
    print('  nodes:', list(nodes.keys()))
    print('  edges:')
    for k, v in edges.items():
        print('   ', k, v)

    # Show enumerated potential routes via recursion (for transparency)
    print('\n[ROUTE ENUMERATION] Enumerating possible simple paths (DFS recursion)')
    paths = enumerate_paths(nodes, edges, start, end, max_depth=10)
    for i, p in enumerate(paths, 1):
        print(f'  Path {i}: {p}')

    print('\n[BUILD LP] Creating PuLP model')
    model = LpProblem('Logistics_Optimization', LpMinimize)
    x = LpVariable.dicts('x', edges.keys(), cat='Binary')

    # Objective
    if objective in ('time', 'fastest'):
        model += lpSum(x[e] * edges[e].get('time', 0) for e in edges), 'Minimize_Time'
    elif objective == 'distance':
        model += lpSum(x[e] * edges[e].get('distance', 0) for e in edges), 'Minimize_Distance'
    else:
        model += lpSum(x[e] * edges[e].get('fuel', 0) for e in edges), 'Minimize_Cost'

    print('  Objective expression set')

    # Flow constraints
    for n in nodes:
        inflow = lpSum(x[(i, j)] for (i, j) in edges if j == n)
        outflow = lpSum(x[(i, j)] for (i, j) in edges if i == n)
        if n == start:
            model += outflow - inflow == 1, f'flow_start_{n}'
        elif n == end:
            model += inflow - outflow == 1, f'flow_end_{n}'
        else:
            model += inflow == outflow, f'flow_int_{n}'

    if via:
        if isinstance(via, list):
            for v in via:
                model += lpSum(x[(i, j)] for (i, j) in edges if j == v) == 1, f'via_{v}'
        else:
            model += lpSum(x[(i, j)] for (i, j) in edges if j == via) == 1, f'via_{via}'

    # Print LP formulation lines by writing to file and then reading it
    try:
        model.writeLP(lp_filename)
        print(f'  LP written to {lp_filename}')
        with open(lp_filename, 'r') as f:
            lp_text = f.read()
        print('\n[LP FILE CONTENT - first 2000 chars]')
        print(lp_text[:2000])
    except Exception as e:
        print('  Could not write/read LP file:', e)

    print('\n[SOLVE] Running solver (CBC)')
    solver = PULP_CBC_CMD(msg=1) if solver_msg else PULP_CBC_CMD(msg=0)
    result = model.solve(solver)
    print('  Solver result code =', result)
    print('  Model status =', model.status)
    if model.status != 1:
        print('  Warning: optimal solution not found')

    print('\n[SOLUTION EXTRACTION] Checking decision variables')
    selected = []
    for e in edges:
        val = x[e].varValue
        print(f'  x{e} = {val}')
        if val == 1:
            selected.append(edges[e])
            print('   -> selected:', edges[e])
    print('\nSelected segments count:', len(selected))
    return selected

print('✓ optimize defined with LP write and recursion enumeration')

✓ optimize defined with LP write and recursion enumeration


In [7]:
def get_route_summary(route):
    print('\n[GET_ROUTE_SUMMARY]')
    if not route:
        print('  route empty')
        return {}
    totals = {'distance': 0, 'time': 0, 'fuel': 0}
    for i, seg in enumerate(route, 1):
        print(f"  Segment {i}: {seg}")
        totals['distance'] += seg.get('distance', 0)
        totals['time'] += seg.get('time', 0)
        totals['fuel'] += seg.get('fuel', 0)
    print('  Totals:', totals)
    return totals

print('✓ get_route_summary defined')

✓ get_route_summary defined


In [8]:
def prepare_vehicles_df(vehicles_df):
    print('\n[PREPARE_VEHICLES_DF] Input DF:')
    print(vehicles_df)
    df = vehicles_df.copy()
    df['base_city'] = df['WarehouseName'].str.lower().str.strip()
    df['vehicle_type'] = df['VehicleType'].str.lower().str.strip()
    df['capacity_kg'] = pd.to_numeric(df['VehicleCapacity'], errors='coerce')
    df['departure_time'] = df['DepartureTime'].str.strip()

    df['vehicle_id_num'] = df.groupby('base_city').cumcount() + 1
    def format_vid(row):
        city_code = row['base_city'][:3].upper()
        type_code = row['vehicle_type'][:3].upper()
        return f"{city_code}-{type_code}-{row['vehicle_id_num']:02d}"
    df['vehicle_id'] = df.apply(format_vid, axis=1)

    df['speed_kmph'] = df['vehicle_type'].map(lambda x: VEHICLE_SPECS.get(x, {}).get('speed', 60))
    df['cost_per_km'] = df['vehicle_type'].map(lambda x: VEHICLE_SPECS.get(x, {}).get('cost', 0.5))
    print('  Prepared DF:')
    print(df[['vehicle_id','base_city','vehicle_type','capacity_kg','speed_kmph','cost_per_km','departure_time']])
    return df[['vehicle_id','base_city','vehicle_type','capacity_kg','speed_kmph','cost_per_km','departure_time']]

print('✓ prepare_vehicles_df defined')

✓ prepare_vehicles_df defined


In [9]:
def assign_vehicles_for_leg(vehicles_df, leg, total_goods, objective='cost'):
    print('\n[ASSIGN_VEHICLES_FOR_LEG]')
    print('  leg:', leg)
    print('  total_goods:', total_goods)
    src = leg['from'].lower()
    available = vehicles_df[vehicles_df['base_city'].str.lower()==src].copy()
    print('  available vehicles:')
    print(available)
    if available.empty:
        print('  no vehicles')
        return None
    model = LpProblem('Vehicle_Assignment', LpMinimize)
    x = LpVariable.dicts('v', available.index, cat='Binary')
    if objective=='time':
        model += lpSum(x[i] * (leg['distance']/available.loc[i,'speed_kmph']) for i in available.index)
    else:
        model += lpSum(x[i] * available.loc[i,'cost_per_km'] * leg['distance'] for i in available.index)
    model += lpSum(x[i]*available.loc[i,'capacity_kg'] for i in available.index) >= total_goods
    model.writeLP('assign_model.lp')
    print('  Written assign_model.lp (first 1000 chars):')
    try:
        print(open('assign_model.lp').read()[:1000])
    except Exception as e:
        print('   could not read lp file:', e)
    res = model.solve(PULP_CBC_CMD(msg=1))
    print('  solve result:', res, ' status:', model.status)
    assigned = []
    remain = total_goods
    for i in available.index:
        if x[i].varValue==1 and remain>0:
            cap = available.loc[i,'capacity_kg']
            load = min(cap, remain)
            depart = available.loc[i,'departure_time'] if pd.notna(available.loc[i,'departure_time']) else '08:00'
            travel_time = leg['distance']/available.loc[i,'speed_kmph']
            assigned.append({'vehicle_id':available.loc[i,'vehicle_id'],'load_kg':load,'departure':depart,'travel_time':travel_time})
            remain -= load
    print('  assigned:', assigned)
    return {'from':leg['from'],'to':leg['to'],'vehicles':assigned}

print('✓ assign_vehicles_for_leg defined')

✓ assign_vehicles_for_leg defined


## Test: run full flow with sample data

In [10]:
# Sample nodes and edges
nodes = {
    'NYC': (40.7128, -74.0060),
    'Philadelphia': (39.9526, -75.1652),
    'Washington': (38.9072, -77.0369),
    'Atlanta': (33.7490, -84.3880)
}
edges = {
    ('NYC','Philadelphia'):{'from':'NYC','to':'Philadelphia','distance':95,'time':1.8,'fuel':47.5,'mode':'truck'},
    ('Philadelphia','Washington'):{'from':'Philadelphia','to':'Washington','distance':140,'time':2.5,'fuel':70.0,'mode':'truck'},
    ('Washington','Atlanta'):{'from':'Washington','to':'Atlanta','distance':640,'time':10.5,'fuel':320.0,'mode':'truck'},
    ('NYC','Washington'):{'from':'NYC','to':'Washington','distance':230,'time':3.8,'fuel':115.0,'mode':'truck'}
}
# Run optimizer; prints recursion enumeration, LP, solver output and selected edges
selected = optimize(nodes, edges, 'NYC', 'Atlanta', objective='cost', lp_filename='test_model.lp', solver_msg=True)
print('\n==> Selected segments:')
for s in selected:
    print('  ', s)


[INPUT]
  start= NYC  end= Atlanta  objective= cost  via= None
  nodes: ['NYC', 'Philadelphia', 'Washington', 'Atlanta']
  edges:
    ('NYC', 'Philadelphia') {'from': 'NYC', 'to': 'Philadelphia', 'distance': 95, 'time': 1.8, 'fuel': 47.5, 'mode': 'truck'}
    ('Philadelphia', 'Washington') {'from': 'Philadelphia', 'to': 'Washington', 'distance': 140, 'time': 2.5, 'fuel': 70.0, 'mode': 'truck'}
    ('Washington', 'Atlanta') {'from': 'Washington', 'to': 'Atlanta', 'distance': 640, 'time': 10.5, 'fuel': 320.0, 'mode': 'truck'}
    ('NYC', 'Washington') {'from': 'NYC', 'to': 'Washington', 'distance': 230, 'time': 3.8, 'fuel': 115.0, 'mode': 'truck'}

[ROUTE ENUMERATION] Enumerating possible simple paths (DFS recursion)
[RECURSION] depth=0 current=NYC path=['NYC']
[RECURSION] depth=1 current=Philadelphia path=['NYC', 'Philadelphia']
[RECURSION] depth=2 current=Washington path=['NYC', 'Philadelphia', 'Washington']
[RECURSION] depth=3 current=Atlanta path=['NYC', 'Philadelphia', 'Washington'

In [11]:
# Route summary of selected route
summary = get_route_summary(selected)
print('\nRoute summary:')
print(summary)


[GET_ROUTE_SUMMARY]
  Segment 1: {'from': 'Washington', 'to': 'Atlanta', 'distance': 640, 'time': 10.5, 'fuel': 320.0, 'mode': 'truck'}
  Segment 2: {'from': 'NYC', 'to': 'Washington', 'distance': 230, 'time': 3.8, 'fuel': 115.0, 'mode': 'truck'}
  Totals: {'distance': 870, 'time': 14.3, 'fuel': 435.0}

Route summary:
{'distance': 870, 'time': 14.3, 'fuel': 435.0}


In [12]:
# Vehicles sample and assignment test
vehicles_raw = pd.DataFrame({
    'WarehouseName':['New York','New York','Philadelphia'],
    'VehicleType':['Truck','Van','Truck'],
    'VehicleCapacity':[5000,2000,5000],
    'DepartureTime':['08:00','09:00','07:00']
})
vehicles = prepare_vehicles_df(vehicles_raw)
leg = {'from':'New York','to':'Philadelphia','distance':95,'time':1.8}
assignment = assign_vehicles_for_leg(vehicles, leg, 4500, objective='cost')
print('\nAssignment result:')
print(assignment)


[PREPARE_VEHICLES_DF] Input DF:
  WarehouseName VehicleType  VehicleCapacity DepartureTime
0      New York       Truck             5000         08:00
1      New York         Van             2000         09:00
2  Philadelphia       Truck             5000         07:00
  Prepared DF:
   vehicle_id     base_city vehicle_type  capacity_kg  speed_kmph  \
0  NEW-TRU-01      new york        truck         5000          80   
1  NEW-VAN-02      new york          van         2000          90   
2  PHI-TRU-01  philadelphia        truck         5000          80   

   cost_per_km departure_time  
0          0.5          08:00  
1          0.4          09:00  
2          0.5          07:00  

[ASSIGN_VEHICLES_FOR_LEG]
  leg: {'from': 'New York', 'to': 'Philadelphia', 'distance': 95, 'time': 1.8}
  total_goods: 4500
  available vehicles:
   vehicle_id base_city vehicle_type  capacity_kg  speed_kmph  cost_per_km  \
0  NEW-TRU-01  new york        truck         5000          80          0.5   
1  NEW-